In [1]:
!pip install conllu
import nltk
import numpy as np
from collections import defaultdict, Counter
from sklearn.metrics import accuracy_score, classification_report

In [2]:
from conllu import parse_incr

def load_ud_conllu(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        data = list(parse_incr(f))
    return [[(tok["form"], tok["upos"]) for tok in sent if tok["upos"]] for sent in data]

train_sents = load_ud_conllu("en_ewt-ud-train.conllu")
dev_sents   = load_ud_conllu("en_ewt-ud-dev.conllu")
test_sents  = load_ud_conllu("en_ewt-ud-test.conllu")

In [3]:
# Extract all unique words and tags
all_words = set()
all_tags = set()

for sentence in train_sents:
    for word, tag in sentence:
        all_words.add(word.lower())  # Lowercase for consistency
        all_tags.add(tag)

# Create word and tag vocabularies
word2idx = {word: idx for idx, word in enumerate(sorted(all_words))}
tag2idx = {tag: idx for idx, tag in enumerate(sorted(all_tags))}
idx2tag = {idx: tag for tag, idx in tag2idx.items()}

# Add unknown token for test words not in training
word2idx['<UNK>'] = len(word2idx)

print(f"Vocabulary size: {len(word2idx)}")
print(f"Number of POS tags: {len(tag2idx)}")
print(f"POS tags: {sorted(all_tags)}")

Vocabulary size: 17114
Number of POS tags: 18
POS tags: ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X', '_']


In [4]:
# Initialize count dictionaries
initial_counts = Counter()
transition_counts = defaultdict(Counter)
emission_counts = defaultdict(Counter)
tag_counts = Counter()

# Count occurrences
for sentence in train_sents:
    prev_tag = None

    for word, tag in sentence:
        word = word.lower()
        tag_counts[tag] += 1
        emission_counts[tag][word] += 1

        if prev_tag is None:
            # First word in sentence
            initial_counts[tag] += 1
        else:
            # Transition from previous tag
            transition_counts[prev_tag][tag] += 1

        prev_tag = tag

print(f"Total tag occurrences: {sum(tag_counts.values())}")

Total tag occurrences: 207228


In [5]:
# Convert counts to probabilities
num_tags = len(tag2idx)

# Initial probabilities
initial_probs = np.zeros(num_tags)
total_initial = sum(initial_counts.values())
for tag, count in initial_counts.items():
    initial_probs[tag2idx[tag]] = count / total_initial

# Transition probabilities (with smoothing)
transition_probs = np.zeros((num_tags, num_tags))
smoothing = 1e-6  # Add-smoothing to avoid zero probabilities

for tag1 in all_tags:
    tag1_idx = tag2idx[tag1]
    total = sum(transition_counts[tag1].values()) + num_tags * smoothing

    for tag2 in all_tags:
        tag2_idx = tag2idx[tag2]
        count = transition_counts[tag1][tag2] + smoothing
        transition_probs[tag1_idx][tag2_idx] = count / total

# Emission probabilities (with smoothing)
vocab_size = len(word2idx)
emission_probs = np.zeros((num_tags, vocab_size))

for tag in all_tags:
    tag_idx = tag2idx[tag]
    total = tag_counts[tag] + vocab_size * smoothing

    for word, word_idx in word2idx.items():
        count = emission_counts[tag][word] + smoothing
        emission_probs[tag_idx][word_idx] = count / total

print("HMM parameters estimated successfully!")

HMM parameters estimated successfully!


In [6]:
def viterbi(sentence, initial_probs, transition_probs, emission_probs, word2idx, tag2idx, idx2tag):
    """
    Viterbi algorithm for finding the most likely tag sequence.

    Args:
        sentence: List of words
        initial_probs: Initial state probabilities
        transition_probs: Tag transition probabilities
        emission_probs: Word emission probabilities
        word2idx, tag2idx, idx2tag: Mappings

    Returns:
        List of predicted tags
    """
    n = len(sentence)
    num_tags = len(tag2idx)

    # Convert words to indices
    word_indices = []
    for word in sentence:
        word = word.lower()
        word_indices.append(word2idx.get(word, word2idx['<UNK>']))

    # Initialize Viterbi matrices
    viterbi_matrix = np.zeros((num_tags, n))
    backpointer = np.zeros((num_tags, n), dtype=int)

    # Initialization (t=0)
    for s in range(num_tags):
        viterbi_matrix[s, 0] = np.log(initial_probs[s] + 1e-10) + \
                                np.log(emission_probs[s, word_indices[0]] + 1e-10)

    # Recursion (t=1 to n-1)
    for t in range(1, n):
        for s in range(num_tags):
            # Find best previous state
            trans_probs = viterbi_matrix[:, t-1] + np.log(transition_probs[:, s] + 1e-10)
            backpointer[s, t] = np.argmax(trans_probs)
            viterbi_matrix[s, t] = trans_probs[backpointer[s, t]] + \
                                    np.log(emission_probs[s, word_indices[t]] + 1e-10)

    # Backtrack to find best path
    best_path = [0] * n
    best_path[-1] = np.argmax(viterbi_matrix[:, -1])

    for t in range(n-2, -1, -1):
        best_path[t] = backpointer[best_path[t+1], t+1]

    # Convert indices to tags
    return [idx2tag[idx] for idx in best_path]

print("Viterbi algorithm implemented!")

Viterbi algorithm implemented!


# we use vetrib to predict

In [7]:
all_true_tags = []
all_predicted_tags = []

print("Evaluating on validation set...")
for i, sentence in enumerate(dev_sents[:3000]):  # Limit to 3000 for speed
    words = [word for word, tag in sentence]
    true_tags = [tag for word, tag in sentence]

    predicted_tags = viterbi(words, initial_probs, transition_probs, emission_probs,
                            word2idx, tag2idx, idx2tag)

    all_true_tags.extend(true_tags)
    all_predicted_tags.extend(predicted_tags)

    if (i + 1) % 200 == 0:
        print(f"Processed {i + 1} sentences...")

# Calculate accuracy
accuracy = accuracy_score(all_true_tags, all_predicted_tags)
print(f"\n{'='*50}")
print(f"Overall Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"{'='*50}")

Evaluating on validation set...
Processed 200 sentences...
Processed 400 sentences...
Processed 600 sentences...
Processed 800 sentences...
Processed 1000 sentences...
Processed 1200 sentences...
Processed 1400 sentences...
Processed 1600 sentences...
Processed 1800 sentences...
Processed 2000 sentences...

Overall Accuracy: 0.8818 (88.18%)


## Test on a single example

In [8]:

test_sentence = test_sents[10]
words = [word for word, tag in test_sentence]
true_tags = [tag for word, tag in test_sentence]

predicted_tags = viterbi(words, initial_probs, transition_probs, emission_probs,
                         word2idx, tag2idx, idx2tag)

print("Sentence:", " ".join(words))
print("\nTrue tags:     ", " ".join(true_tags))
print("Predicted tags:", " ".join(predicted_tags))

Sentence: I doubt the very few who actually read my blog have not come across this yet , but I figured I would put it out there anyways .

True tags:      PRON VERB DET ADV ADJ PRON ADV VERB PRON NOUN AUX PART VERB ADP PRON ADV PUNCT CCONJ PRON VERB PRON AUX VERB PRON ADV ADV ADV PUNCT
Predicted tags: PRON VERB DET ADV ADJ PRON ADV VERB PRON NOUN AUX PART VERB ADP DET ADV PUNCT CCONJ PRON VERB PRON AUX VERB PRON ADP PRON INTJ PUNCT


# Predict tags for all test sentences

In [9]:
all_true_tags = []
all_predicted_tags = []

print("Evaluating on test set...")
for i, sentence in enumerate(test_sents[:3000]):  # Limit to 1000 for speed
    words = [word for word, tag in sentence]
    true_tags = [tag for word, tag in sentence]

    predicted_tags = viterbi(words, initial_probs, transition_probs, emission_probs,
                            word2idx, tag2idx, idx2tag)

    all_true_tags.extend(true_tags)
    all_predicted_tags.extend(predicted_tags)

    if (i + 1) % 200 == 0:
        print(f"Processed {i + 1} sentences...")

# Calculate accuracy
accuracy = accuracy_score(all_true_tags, all_predicted_tags)
print(f"\n{'='*50}")
print(f"Overall Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"{'='*50}")

Evaluating on test set...
Processed 200 sentences...
Processed 400 sentences...
Processed 600 sentences...
Processed 800 sentences...
Processed 1000 sentences...
Processed 1200 sentences...
Processed 1400 sentences...
Processed 1600 sentences...
Processed 1800 sentences...
Processed 2000 sentences...

Overall Accuracy: 0.8847 (88.47%)


In [10]:
for sentence in test_sents:
    words = [w[0] for w in sentence]
    true_tags = [t for (_, t) in sentence]

    predicted_tags = viterbi(words, initial_probs, transition_probs, emission_probs,
                            word2idx, tag2idx, idx2tag)
    all_true_tags.extend(true_tags)
    all_predicted_tags.extend(predicted_tags)



from sklearn.metrics import classification_report

print(classification_report(all_true_tags, all_predicted_tags, digits=3))

              precision    recall  f1-score   support

         ADJ      0.888     0.879     0.883      3576
         ADP      0.906     0.937     0.921      4050
         ADV      0.863     0.848     0.856      2382
         AUX      0.899     0.970     0.933      3086
       CCONJ      0.964     0.984     0.974      1472
         DET      0.907     0.973     0.939      3794
        INTJ      0.330     0.868     0.478       242
        NOUN      0.870     0.874     0.872      8246
         NUM      0.889     0.679     0.770      1084
        PART      0.885     0.949     0.916      1298
        PRON      0.932     0.965     0.948      4330
       PROPN      0.860     0.561     0.679      4150
       PUNCT      0.964     0.976     0.970      6192
       SCONJ      0.775     0.727     0.750       768
         SYM      0.630     0.814     0.710       226
        VERB      0.907     0.884     0.896      5212
           X      0.072     0.476     0.125        84
           _      0.900    

In [12]:
import json

# Load the JSON file
with open("movies.json", "r", encoding="utf-8") as f:
    script_data = json.load(f)

In [24]:
print("First few entries in script_data:", script_data[:3])

First few entries in script_data: [{'Name': 'I am Sam', 'Script': '"I AM SAM"\r\n\r\n                                      Screenplay by\r\n\r\n                                     Kristine Johnson\r\n\r\n                                      Jessie Nelson\r\n\r\n                                           2001\r\n\r\n                                      SHOOTING DRAFT\r\n\r\n\r\n\r\n            INT. STARBUCKS - 7:30 A.M.\r\n\r\n            We\'re watching a pair of hands arrange white sugar packets,\r\n            blue Equal packets, and pink Sweet and Low into small\r\n            containers.  With precision and lightning speed, the mixed up\r\n            colors and crumpled packets are transformed into neat little\r\n            color-coded rows.  Wait, this container has three Equals and\r\n            four Sweet \'n\' lows.  The hand quickly plucks the mutant\r\n            Sweet \'n\' Low.  There.  Symmetry.\r\n\r\n            We move up those hands and meet SAM DAWSON as he sur

In [28]:
import nltk
nltk.download("punkt", download_dir="/usr/local/nltk_data")

[nltk_data] Downloading package punkt to /usr/local/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [30]:
nltk.data.path.append("/usr/local/nltk_data")

In [33]:
import re

def simple_tokenize(text):
    # Basic word tokenizer: splits on spaces and punctuation
    return re.findall(r"\b\w+\b|[^\w\s]", text)

In [37]:
for tokens in tokenized_sentences:
    predicted_tags = viterbi(words, initial_probs, transition_probs, emission_probs,
                            word2idx, tag2idx, idx2tag)
    print(list(zip(tokens, predicted_tags)))

def is_valid_line(line):
    # Skip all-uppercase lines and short fragments
    return line.strip() and not line.isupper() and len(line.split()) > 2

tokenized_sentences = []
for entry in script_data:
    if isinstance(entry, dict) and "Script" in entry:
        script = entry["Script"]
        lines = script.split("\n")
        for line in lines:
            if is_valid_line(line):
                tokens = simple_tokenize(line)  # or nltk.word_tokenize if punkt works
                tokenized_sentences.append(tokens)

[('"', 'PRON'), ('I', 'NOUN'), ('AM', 'CCONJ'), ('SAM', 'AUX'), ('"', 'ADJ'), ('Screenplay', 'ADP'), ('by', 'PROPN'), ('Kristine', 'PUNCT'), ('Johnson', 'PUNCT'), ('Jessie', 'CCONJ'), ('Nelson', 'VERB'), ('2001', 'DET'), ('SHOOTING', 'ADJ'), ('DRAFT', 'NOUN'), ('INT', 'CCONJ'), ('.', 'VERB'), ('STARBUCKS', 'VERB'), ('-', 'PART'), ('7', 'VERB'), (':', 'PUNCT')]
[('THE', 'PRON'), ('BOOK', 'NOUN'), ('OF', 'CCONJ'), ('ELI', 'AUX'), ('Written', 'ADJ'), ('by', 'ADP'), ('Gary', 'PROPN'), ('Whitta', 'PUNCT'), ('June', 'PUNCT'), ('21', 'CCONJ'), (',', 'VERB'), ('2007', 'DET'), ('A', 'ADJ'), ('civilization', 'NOUN'), ('is', 'CCONJ'), ('destroyed', 'VERB'), ('only', 'VERB'), ('when', 'PART'), ('its', 'VERB'), ('gods', 'PUNCT')]
[('"', 'PRON'), ('s', 'NOUN'), ('e', 'CCONJ'), ('x', 'AUX'), (',', 'ADJ'), ('l', 'ADP'), ('i', 'PROPN'), ('e', 'PUNCT'), ('s', 'PUNCT'), (',', 'CCONJ'), ('a', 'VERB'), ('n', 'DET'), ('d', 'ADJ'), ('v', 'NOUN'), ('i', 'CCONJ'), ('d', 'VERB'), ('e', 'VERB'), ('o', 'PART'), (

In [38]:
print("Number of tokenized sentences:", len(tokenized_sentences))
print("First few sentences:", tokenized_sentences[:3])

Number of tokenized sentences: 2979959
First few sentences: [['We', "'", 're', 'watching', 'a', 'pair', 'of', 'hands', 'arrange', 'white', 'sugar', 'packets', ','], ['blue', 'Equal', 'packets', ',', 'and', 'pink', 'Sweet', 'and', 'Low', 'into', 'small'], ['containers', '.', 'With', 'precision', 'and', 'lightning', 'speed', ',', 'the', 'mixed', 'up']]


In [54]:
for sentence in tokenized_sentences[:10]:

  predicted_tags = viterbi(sentence, initial_probs, transition_probs, emission_probs,
                             word2idx, tag2idx, idx2tag)

    # Print first 5 tokens only
  print("Sentence:", " ".join(sentence[:5]))
  print("Tags:", " ".join(predicted_tags[:5]))
  print("=" * 50)




Sentence: We ' re watching a
Tags: PRON PUNCT AUX VERB DET
Sentence: blue Equal packets , and
Tags: ADJ ADJ NOUN PUNCT CCONJ
Sentence: containers . With precision and
Tags: NOUN PUNCT ADP PROPN CCONJ
Sentence: colors and crumpled packets are
Tags: NOUN CCONJ VERB NOUN AUX
Sentence: color - coded rows .
Tags: PROPN PUNCT DET NOUN PUNCT
Sentence: four Sweet ' n '
Tags: NUM ADJ PUNCT PROPN PUNCT
Sentence: Sweet ' n ' Low
Tags: ADJ PUNCT PROPN PUNCT ADJ
Sentence: We move up those hands
Tags: PRON VERB ADP DET NOUN
Sentence: domain . Something about him
Tags: NOUN PUNCT PRON ADP PRON
Sentence: uniquely handsome . But it
Tags: ADV ADJ PUNCT CCONJ PRON
